# 6 setup a cronjob to back up mongodb

change ${PJ_ROOT} to your directory.

In [ ]:
export PJ_ROOT="${HOME}/core"
cd ${PJ_ROOT};pwd

example)
```
/Users/user/core
```

## load environment variables

In [ ]:
source ${PJ_ROOT}/docs/environments/azure_aks/env

## create a storage account

In [ ]:
az storage account create --name ${STORAGE_ACCOUNT} --resource-group ${AKS_RG} --location ${REGION} --sku Standard_LRS --encryption blob

In [ ]:
export ACCOUNT_KEY=$(az storage account keys list --account-name ${STORAGE_ACCOUNT} --resource-group ${AKS_RG} --output json | jq .[0].value -r)

## create storage container of Azure Blob Storage

In [ ]:
az storage container create --name ${STORAGE_CONTAINER} --account-name ${STORAGE_ACCOUNT} --account-key ${ACCOUNT_KEY}

In [ ]:
az storage blob list --container-name ${STORAGE_CONTAINER} --account-name ${STORAGE_ACCOUNT} --account-key ${ACCOUNT_KEY}

example)
```json
[]
```

## start backup job to check the environment

In [ ]:
envsubst < mongodb/simple-dump-job.yaml | kubectl apply -f -

In [ ]:
kubectl get jobs -l job=simple-dump-job

example)
```
NAME              COMPLETIONS   DURATION   AGE
simple-dump-job   1/1           5s         27s
```

In [ ]:
kubectl get pods -l job=simple-dump-job

example)
```
NAME                    READY   STATUS      RESTARTS   AGE
simple-dump-job-7td2x   0/1     Completed   0          45s
```

In [ ]:
kubectl logs $(kubectl get pods -l job=simple-dump-job -o template --template "{{(index .items 0).metadata.name}}")

example)
```
2019/06/07 08:48:32 [   INFO] __main__ - start main
2019/06/07 08:48:32 [   INFO] src.mongodb - start dump, endpoint=rs0/mongodb-client:27017
2019/06/07 08:48:32 [   INFO] src.mongodb - exec dump command, dump_dir=/opt/mongodump_20190607174832JST
2019/06/07 08:48:33 [   INFO] src.mongodb - 
2019/06/07 08:48:33 [   INFO] src.mongodb - 2019-06-07T08:48:33.758+0000	writing admin.system.version to 
2019-06-07T08:48:33.760+0000	done dumping admin.system.version (1 document)
2019-06-07T08:48:33.760+0000	writing iotagentul.groups to 
2019-06-07T08:48:33.760+0000	writing orion.entities to 
2019-06-07T08:48:33.761+0000	done dumping iotagentul.groups (0 documents)
2019-06-07T08:48:33.761+0000	done dumping orion.entities (0 documents)

2019/06/07 08:48:33 [   INFO] src.mongodb - compress dump dir, dump_dir=/opt/mongodump_20190607174832JST
2019/06/07 08:48:33 [   INFO] src.mongodb - finish dump, dump_file=/opt/mongodump_20190607174832JST.tar.gz
2019/06/07 08:48:33 [   INFO] src.azure_blob_storage - start upload, container=mongodump, file=/opt/mongodump_20190607174832JST.tar.gz
2019/06/07 08:48:33 [   INFO] azure.storage.common.storageclient - Client-Request-ID=083ca8f2-8901-11e9-b537-a6ad4145284e Outgoing request: Method=PUT, Path=/mongodump/mongodump_20190607174832JST.tar.gz, Query={'timeout': None}, Headers={'x-ms-blob-type': 'BlockBlob', 'x-ms-lease-id': None, 'If-Modified-Since': None, 'If-Unmodified-Since': None, 'If-Match': None, 'If-None-Match': None, 'Content-Length': '700', 'x-ms-version': '2018-11-09', 'User-Agent': 'Azure-Storage/2.0.0-2.0.1 (Python CPython 3.7.3; Linux 4.15.0-1045-azure)', 'x-ms-client-request-id': '083ca8f2-8901-11e9-b537-a6ad4145284e', 'x-ms-date': 'Fri, 07 Jun 2019 08:48:33 GMT', 'Authorization': 'REDACTED'}.
2019/06/07 08:48:33 [   INFO] azure.storage.common.storageclient - Client-Request-ID=083ca8f2-8901-11e9-b537-a6ad4145284e Receiving Response: Server-Timestamp=Fri, 07 Jun 2019 08:48:33 GMT, Server-Request-ID=6829bef5-e01e-003f-760d-1d20d7000000, HTTP Status Code=201, Message=Created, Headers={'content-length': '0', 'content-md5': 'jLwedUl693uDUHgfnlyUyg==', 'last-modified': 'Fri, 07 Jun 2019 08:48:33 GMT', 'etag': '"0x8D6EB24EC81FEAC"', 'server': 'Windows-Azure-Blob/1.0 Microsoft-HTTPAPI/2.0', 'x-ms-request-id': '6829bef5-e01e-003f-760d-1d20d7000000', 'x-ms-version': '2018-11-09', 'x-ms-request-server-encrypted': 'true', 'date': 'Fri, 07 Jun 2019 08:48:33 GMT'}.
2019/06/07 08:48:33 [   INFO] src.azure_blob_storage - finish upload, last_modified=2019-06-07 08:48:33+00:00
2019/06/07 08:48:33 [   INFO] __main__ - finish main
```

In [ ]:
az storage blob list --container-name ${STORAGE_CONTAINER} --account-name ${STORAGE_ACCOUNT} --account-key ${ACCOUNT_KEY} -o table

example)
```
Name                                Blob Type    Blob Tier    Length    Content Type              Last Modified              Snapshot
----------------------------------  -----------  -----------  --------  ------------------------  -------------------------  ----------
mongodump_20190607174832JST.tar.gz  BlockBlob                 700       application/octet-stream  2019-06-07T08:48:33+00:00
```

In [ ]:
kubectl delete job simple-dump-job

## start cronjob

In [ ]:
envsubst < mongodb/simple-dump-cronjob.yaml | kubectl apply -f -

In [ ]:
kubectl get cronjobs -l job=simple-dump-cronjob

example)
```
NAME                  SCHEDULE         SUSPEND   ACTIVE   LAST SCHEDULE   AGE
simple-dump-cornjob   0 0-23/3 * * *   False     0        <none>          12s
```

**the backup job will start every day at 00:00, 03:00, 06:00, 09:00, 12:00, 15:00, 18:00, 21:00**